# How to exploit data on Pangeo

## Snow and Cloud Cover Example
The executable content of this notebook is adapted from https://github.com/EO-College/cubes-and-clouds/blob/main/lectures/3.1_data_processing/exercises/_alternatives/31_data_processing_stac.ipynb

## Authors & Contributors
### Authors
- Michele Claus, Eurac Research (Italy), [clausmichele](https://github.com/clausmichele) (original author)
- Pier Lorenzo Marasco, Provare LTD (UK), [@pl-marasco](https://github.com/pl-marasco) (author of the conversion)

### Contributors
- Alejandro Coca-Castro, The Alan Turing Institure, [acocac](https://github.com/acocac) (author)
- Anne Fouilloux, Simula Research Laboratory (Norway), [@annefou](https://github.com/annefou)
- Justus Magin, UMR-LOPS CNRS(France), [@justusmagin](https://github.com/justusmagin)
- Tina Odaka, UMR-LOPS Ifremer (France), [@tinaok](https://github.com/tinaok)

## Introduction
In this exercise, we will build a complete the same EO workflow as OpenEO using cloud provided data (STAC Catalogue), processing it locally; from data access to obtaining the result.

We are going to follow these steps in our analysis:

- Load satellite collections
- Specify the spatial, temporal extents and the features we are interested in
- Process the satellite data to retrieve snow cover information
- Aggregate information in data cubes
- Visualize and analyse the results

### Relevant resources

* More information on Pangeo can be found here: https://pangeo.io/
* More information on the STAC specification can be found here: https://stacspec.org/


## Import libraries

In [ ]:
# Data Manipulation and Analysis Libraries
import pandas as pd  
import numpy as np 

# Geospatial Data Handling Libraries
import geopandas as gpd 
from shapely.geometry import mapping  
import pyproj

# Multidimensional and Satellite Data Libraries
import xarray as xr 
import rioxarray as rio
import stackstac

# Data Visualization Libraries
import holoviews as hv
import hvplot.xarray
import hvplot.pandas

# Data parallelization and distributed computing libraries
import dask
from dask.distributed import Client, progress, LocalCluster

# STAC Catalogue Libraries
import pystac_client

## Connect to the Dask cluster

Dask cluster can be deployed on HPC clusters, cloud computing services, or on a local machine. More on how to deploy over different platforms can be found here: https://docs.dask.org/en/stable/deploying.html
Here we creates a Dask client, which is essential for managing and executing parallel computations efficiently in the subsequent parts of the notebook. 

<div class="alert alert-block alert-danger"> WARNING !
If you are going to use dask_gateway (on pangeo-EOSC), please activate next cell.  If you are using your local PC, or you do not use dask_gateway, leave it as it is.
</div>

<div class="alert alert-block alert-danger"> WARNING !
Please deactivate cell below if you use dask_gateway (on pangeo-EOSC).  If you are using your local PC, or you do not use dask_gateway, leave it as it is.
</div>

In [ ]:
cluster = None

### Get a client from the Dask  Cluster

As stated above,creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [ ]:
from distributed import Client

if cluster:
    client = Client(cluster)  # create a dask Gateway cluster
else:
    cluster = LocalCluster()
    client = Client(cluster)  # create a local dask cluster on the machine.
client

Let's setup the Dask Dashboard with your new cluster, as explained in the former section (Parallel computing with Dask)
Reminder: 
- ***If you use Dask-Gateway: just click on the link to open the dashboard into another tab. Then copy and pastthe link of web site appearing to the dask lab - extension***
-  ...


## Load data

### Catchment outline

We will use the catchment as our area of interest (AOI) for the analysis. The catchment is defined by a polygon, which we will load from a GeoJSON file. 
The GeoJSON file contains the geometry of the catchment in the WGS84 coordinate reference system (EPSG:4326) and that has to be defined. 

In [ ]:
aoi = gpd.read_file('../data/catchment_outline.geojson', crs="EPGS:4326")
aoi_geojson = mapping(aoi.iloc[0].geometry)

### Satellite collections

#### Search for satellite data using STAC

We will utilize the STAC API to search for satellite data in this exercise, specifically leveraging the API provided by AWS/Element84. The STAC API operates as a RESTful service, enabling the querying of satellite data with various filters such as spatial range, time period, and other specific metadata. This API is constructed based on the STAC specification, a collaborative, community-driven standard aimed at enhancing the discoverability and usability of satellite data. Numerous data providers, including AWS, Google Earth Engine, and Planet (Copernicus Data Space Ecosystem (CDSE) is coming soon**), among others, have implemented the STAC API, exemplifying its widespread adoption and utility in accessing diverse satellite datasets.

For the purposes of this exercise, we will limit the search to the Sentinel 2 L2A collection, which is a collection of Sentinel 2 data that has been processed to surface reflectance (Top Of Canopy).
We will also limit the search to the time period between 1st February 2019 and 10th June 2019 and to the extent of the catchment.

** at the moment of writing the STAC catalog of the CDSE is not yet fully operational.

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
items = catalog.search(
    intersects=aoi_geojson,
    collections=["sentinel-2-l2a"],
    datetime="2019-02-01/2019-06-10"
).item_collection()
len(items)

#### Get bands information
As the original data provides bands with different names than the original Sentinel 2 bands, we need to get the information about the bands.

In [ ]:
# Get bands information
# selected_item = items[1]
# for key, asset in selected_item.assets.items():
#     print(f"{key}: {asset.title}")

#### Load data
We will use the stackstac library to load the data. The stackstac library is a library that allows loading data from a STAC API into an xarray dataset.
Here we will load the green and swir16 bands, which are the bands we will use to calculate the snow cover. We will also load the scl band, which is the scene classification layer, which we will use to mask out clouds.
Spatial resolution of 20m is selected for the analysis. The data is loaded in chunks of 2048x2048 pixels.

[Stackstac](https://stackstac.readthedocs.io/en/latest/) is not the only way to create a xarray dataset from a STAC API. Other libraries can be used, such as [xpystac](https://github.com/stac-utils/xpystac) or [odc.stac](https://github.com/opendatacube/odc-stac). The choice of the library depends on the use case and specific needs.

In [ ]:
ds = stackstac.stack(items,
                    bounds_latlon=aoi.iloc[0].geometry.bounds,
                    resolution=20,
                    chunksize=2048,
                    assets=['green', 'swir16', 'scl'])

## Calculate snow cover

We will calculate the Normalized Difference Snow Index (NDSI) to calculate the snow cover. The NDSI is calculated as the difference between the green and the swir16 bands divided by the sum of the green and the swir16 bands.

For a matter of clarity we will define the green and the swir16 bands as variables. Other approaches can be used to manage the data, but this is the one we will use in this exercise.

In [ ]:
green = ds.sel(band='green')
swir = ds.sel(band='swir16')
scl = ds.sel(band='scl')

Let's compute the NDSI and mask out the clouds.

In [ ]:
ndsi = (green - swir) / (green + swir)
ndsi

<div class="alert alert-block alert-info">Dask Method Differences: `.compute()` vs `.persist()`

Dask provides two primary methods for executing computations: `.compute()` and `.persist()`. Below is an overview of each method and their typical use cases.

#### `.compute()`
- **Functionality**: Executes the Dask computation and blocks until the result is available. It then collects and returns the final result to the local process.
- **Use Case**: Invoke `.compute()` when you need to bring the computed result into your local memory. It is typically used as the final step in a Dask workflow after all transformations and computations have been defined.
- **Evaluation**: Eager - runs immediately and provides results.

#### `.persist()`
- **Functionality**: Begins computing the result in the background while immediately returning a new Dask object that represents the ongoing computation.
- **Use Case**: Utilize `.persist()` in a distributed environment when working with large datasets or complex computations that have expensive intermediate steps. This will keep the intermediate results in the cluster’s distributed memory, improving performance for subsequent computations.
- **Evaluation**: Lazy - computations are started but the method returns a reference to the future result without waiting for the completion.

Each method plays a crucial role in optimizing and managing the execution of large-scale computations using Dask, particularly when balancing memory usage and computational efficiency in a distributed setting.
</div>

In [ ]:
ndsi = ndsi.persist()

We will mask out the clouds, which are identified by the values 8 and 9 in the scene classification layer (scl). The scl contains information about the type of land cover. We will mask out the clouds, which are identified by the values 8 and 9 in the scl layer.

More detailed info can be found here: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm-overview

In [ ]:
snow = xr.where((ndsi > 0.42) & ~np.isnan(ndsi), 1, ndsi)
snowmap = xr.where((snow <= 0.42) & ~np.isnan(snow), 0, snow)

As the SCL layer contains information about the type of land cover, we will mask out the clouds, which are identified by the values 8 and 9 in the scl layer.

In [ ]:
mask = np.logical_not(scl.isin([8, 9, 3]))  # alternative you can use the mask = (scl != 8) & (scl != 9) & (scl != 3) 
snow_cloud = xr.where(mask, snowmap, 2)

## Process snow cover data

### Mask data

As we are only interested to the snow cover in the catchment, we will mask out the data outside the catchment. To achieve it we need to convert the catchment geometry to the same coordinate reference system as the data. The data is in the UTM32N coordinate reference system (EPSG:32632).

In [ ]:
aoi_utm32 = aoi.to_crs(epsg=32632)
geom_utm32 = aoi_utm32.iloc[0]['geometry']

As we are going to use the `RioXarray` library to mask out the data, we need to add some more information to the data. The RioXarray library is a library that allows to manipulate geospatial data in xarray datasets. Underneath it uses the rasterio library that is a library built on top of GDAL.

We need first to specify the coordinate reference system and the nodata value. Both information can be found in the metadata of the data but we need to reinforce it so that `RioXarray` can use it.

In [ ]:
snow_cloud.rio.write_crs("EPSG:32632", inplace=True)
snow_cloud.rio.set_nodata(np.nan, inplace=True)

Let's clip the snow_cloud object using the catchment geometry in the UTM32N coordinate reference system.

In [ ]:
snowmap_clipped = snow_cloud.rio.clip([geom_utm32])

It's time to persist the data in memory. We will use the persist method to load the data in memory and keep it there until the end of the analysis.

In [ ]:
clipped_date = snowmap_clipped.persist()

### Aggregate data

Data aggregation is a very important step in the analysis. It allows to reduce the amount of data and to make the analysis more efficient. Moreover as in this case we are going to aggregate the date to daily values, this will allow use to compute statistic on the data at the basin scale later on.

The `groupby` method allows to group the data by a specific dimension. We will group the data by the time dimension, aggregating to the date and removing the time information, once the group is obtained we will aggregate the data by taking the maximum value.

In [ ]:
clipped_date.time

In [ ]:
clipped_date = snowmap_clipped.groupby(snowmap_clipped.time.dt.floor('D')).max(skipna=True)

As the data has been aggregated to daily values, we need to rename the floor method to something more meaningful as date.

In [ ]:
clipped_date = clipped_date.rename({'floor': 'date'})

In [ ]:
clipped_date = clipped_date.persist()

### Visualize data
We will use the `hvplot` library to visualize the data. The library allows to visualize data in `xarray` datasets. It is based on the holoviews library, which is a library that allows to visualize multidimensional data.
To visualize the data on a map, we need to specify the coordinate reference system of the data. The data is in the UTM32N coordinate reference system (EPSG:32632). This will allow the library to project the data on a map.
More info on the hvplot library can be found here: https://hvplot.holoviz.org/

In [ ]:
clipped_date.hvplot.image(
    x='x',
    y='y',
    groupby='date',
    crs=pyproj.CRS.from_epsg(32632),
    cmap='Pastel2',
    clim=(-1, 2),
    frame_width=500,
    frame_height=500,
    title='Snowmap',
    geo=True, tiles='OSM')

### Calculate snow cover with apply_ufunc

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Calculate snow cover using Xarray's apply_ufunc </b>
    <br>
    <ul>
        <li>The procedure for computing snow cover can also be summed up as following python function.  
</li> 
        <li>We first verify that Green, swir16 and scr are in the order of 0,1,2 th variable in band variable.   Then we simply copy and past all the python codes in a function.  </li>
            </ul>
</div>

In [ ]:
def calculate_ndsi_snow_cloud(data):
    green = data[0]
    swir = data[1]
    scl = data[2]
    ndsi = (green - swir) / (green + swir)
    ndsi_mask = ( ndsi > 0.4 )& ~np.isnan(ndsi)
    snow = np.where(ndsi_mask, 1, ndsi)
    snowmap = np.where((snow <= 0.42) & ~np.isnan(snow), 0, snow)
    mask = ~( (scl == 8) | (scl == 9) | (scl == 3) )
    snow_cloud = np.where(mask, snowmap, 2)
    return snow_cloud

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Apply mask then persist the data, then apply_ufunc to perform computation.   </b>
    <br>
    <ul>
        <li>The masking procedure can also applied before the computation.     
</li> 
        <li>Xarray's apply_ufunc is passed to each chunk of Xarray.DataArray     
</li> 
        <li>chunksize for stackstac is specified so that it is only chunked (sliced) in time. All datas from band, x and y are in one chunk (slice) 
</li>         
    </ul>
</div>

In [ ]:
%%time
da = stackstac.stack(items,
                    bounds_latlon=aoi.iloc[0].geometry.bounds,
                    resolution=20,
                    chunksize=(1,-1,-1,-1),
                    assets=['green', 'swir16', 'scl'])
#Mask data
geom_utm32 = aoi.to_crs(epsg=32632).iloc[0]['geometry']
da.rio.write_crs("EPSG:32632", inplace=True)
da.rio.set_nodata(np.nan, inplace=True)
da = da.rio.clip([geom_utm32])

snow_cloud_clipped=xr.apply_ufunc(
    calculate_ndsi_snow_cloud
    ,da
    ,input_core_dims=[["band","y","x"]]
    ,output_core_dims=[["y","x"]]
    ,exclude_dims=set(["band"])
    ,vectorize=True
    ,dask="parallelized"
    ,output_dtypes=[da.dtype]
    ).assign_attrs({'long_name': 'snow_cloud'}).to_dataset(name='snow_cloud')

snow_cloud_clipped
#snow_cloud_clipped_date = snow_cloud_clipped.groupby(snow_cloud_clipped.time.dt.floor('D')).max(skipna=True).rename({'floor': 'date'})
#snow_cloud_clipped_date

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Inspect the data dimentions! </b>
    <br>
    <ul>
        <li>How did changed from input (da) to output(snow_cloud_clipped)?
        </li>
        <li>What is setted as input_core_dims?    
</li> 
        <li>What is setted as output_core_dims?   
</li> 
        <li>What is setted as exclude_dims? 
</li> 
        <li>Did you see 'time' dimension?
</li> 
        <li>We will get back to apply_ufunc with next OpenEO example.  
</li> 
    </ul>
</div>

## Compute statistics

Our objective is to monitor a specific area over a period of time, ensuring the data quality meets our standards. To achieve this, we determine the proportion of clouds in the watershed at each interval. This cloud coverage data serves to refine our timeline: we exclude any interval where cloud cover exceeds 25%.

Our primary focus is on quantifying the Snow Covered Area (SCA) in the watershed. We tally the number of pixels depicting snow for each interval and calculate the SCA by multiplying the snowy pixels by the pixel's area. To ascertain the extent of snow coverage, we compare the snowy pixel count to the watershed's total pixel count, resulting in the percentage of the area that is snow-laden. This percentage is crucial to our analysis.

We need to gather the total pixel counts for the entire watershed, as well as those specific to cloud and snow coverages.

In [ ]:
# number of cloud pixels
cloud = xr.where(clipped_date == 2, 1, np.nan).count(dim=['x', 'y']).persist()

In [ ]:
# number of all pixels per each single date
aot_total = clipped_date.count(dim=['x', 'y']).persist()

In [ ]:
# Cloud fraction per each single date expressed in % 
cloud_fraction = (cloud / aot_total * 100).persist()

In [ ]:
# Visualize cloud fraction
cloud_fraction.hvplot.line(title='Cloud cover %', ylabel="&") * hv.HLine(25).opts(
    color='red',
    line_dash='dashed',
    line_width=2.0,
)

We are going to get the same information for the snow cover.

In [ ]:
snow = xr.where(clipped_date == 1, 1, np.nan).count(dim=['x', 'y']).persist()

In [ ]:
snow_fraction = (snow / aot_total * 100).persist()

In [ ]:
# visualize snow fraction
snow_fraction.hvplot.line(title='Snow cover area (%)', ylabel="%")

In [ ]:
# mask out cloud fraction > 30% 
masked_cloud_fraction = cloud_fraction < 30

In [ ]:
snow_selected = snow_fraction.sel(date=masked_cloud_fraction)

In [ ]:
snow_selected.name = 'SCA'
snow_selected.hvplot.line(title="Snow fraction")

Let's compare the date with the discharge data.

In [ ]:
discharge_ds = pd.read_csv('../data/ADO_DSC_ITH1_0025.csv', sep=',', index_col='Time', parse_dates=True)

Let's refine a little bit the data so that we can compare it with the snow cover data.

In [ ]:
start_date = pd.to_datetime("2019/02/01")
end_date = pd.to_datetime("2019/06/30")
# filter discharge data to start and end dates
discharge_ds = discharge_ds.loc[start_date:end_date]

In [ ]:
discharge_ds.discharge_m3_s.hvplot(title='Discharge volume', ylabel='Discharge (m$^3$/s)') * snow_selected.hvplot(ylabel='Snow cover area (%)')  

## Conclusion

In this analysis, we have comprehensively examined the features, capabilities, and limitations of two prominent geospatial data processing frameworks: OpenEO and Pangeo. OpenEO offers a unified API that simplifies the process of accessing and processing earth observation data across various backends, allowing users to interact with different data sources seamlessly. Its standardized interface is a strong asset, making it accessible to a wide range of users, from researchers to application developers.

On the other hand, Pangeo excels in facilitating big data geoscience. Its robust ecosystem, built around existing Python libraries like Dask and Xarray, makes it a powerful tool for large-scale data analysis and visualization. Pangeo’s community-driven approach and open-source nature foster collaboration and innovation, promoting a dynamic and adaptable framework.

Each platform has its own set of advantages and constraints. OpenEO simplifies interoperability and enhances accessibility, making it particularly beneficial for users who wish to work with diverse data sources without delving deeply into the complexities of each backend. Pangeo, with its emphasis on leveraging existing Python tools and libraries, is particularly potent for those comfortable with Python and seeking to perform extensive, scalable analyses.

Choosing between OpenEO and Pangeo ultimately depends on the specific requirements and constraints of a project. Considerations such as the user's familiarity with Python, the necessity for interoperability across various data backends, and the scale of data processing required should guide the decision-making process.